This is kind of a mess of different model configurations and debugging of mysterious MXNet errors that I still haven't figured out

In [284]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import optimizer
from mxnet import autograd as ag
from mxnet.gluon import HybridBlock
from mxnet.gluon.data import vision
from gluoncv.utils import makedirs, TrainingHistory
import gluoncv
import numpy as np
import time

In [323]:
### AlexNet and CIFAR100 
# Net

class AlexNet(nn.Block):
    r"""AlexNet model from the `"One weird trick..." `_ paper.

    Parameters
    ----------
    classes : int, default 1000
        Number of classes for the output layer.
    """
    def __init__(self, classes=1000, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        self.blk = nn.Sequential()
        self.blk.add(nn.Conv2D(64, kernel_size=11, strides=4,
                                    padding=5, activation='relu', in_channels=3,layout="NHWC"))
        self.blk.add(nn.MaxPool2D(pool_size=3, strides=2, padding=1))
        self.blk.add(nn.Conv2D(256, kernel_size=5, padding=2,
                                    activation='relu',layout="NHWC"))
        self.blk.add(nn.MaxPool2D(pool_size=3, strides=2, padding=1))
        self.blk.add(nn.Conv2D(384, kernel_size=3, padding=1,
                                    activation='relu', layout="NHWC"))
        self.blk.add(nn.Conv2D(256, kernel_size=3, padding=1,
                                    activation='relu', layout="NHWC"))
        self.blk.add(nn.Conv2D(256, kernel_size=3, padding=1,
                                    activation='relu', layout="NHWC"))
        self.blk.add(nn.MaxPool2D(pool_size=3, strides=2, padding=1))
        self.blk.add(nn.Flatten())
        self.blk.add(nn.Dense(4096, activation='relu'))
        self.blk.add(nn.Dropout(0.5))
        self.blk.add(nn.Dense(4096, activation='relu'))
        self.blk.add(nn.Dropout(0.5))
        self.blk.add(nn.Dense(classes))
        
    def forward(self, x):
        return self.blk(x)

In [416]:
net = nn.Sequential()
net.add(nn.Conv2D(64, kernel_size=11, strides=4,
                            padding=5, activation='relu', in_channels=3,layout="NHWC"))
net.add(nn.MaxPool2D(pool_size=3, strides=2, padding=1))
net.add(nn.Conv2D(256, kernel_size=5, padding=2,
                            activation='relu',layout="NHWC"))
net.add(nn.MaxPool2D(pool_size=3, strides=2, padding=1))
net.add(nn.Conv2D(384, kernel_size=3, padding=1,
                            activation='relu', layout="NHWC"))
net.add(nn.Conv2D(256, kernel_size=3, padding=1,
                            activation='relu', layout="NHWC"))
net.add(nn.Conv2D(256, kernel_size=3, padding=1,
                            activation='relu', layout="NHWC"))
net.add(nn.MaxPool2D(pool_size=3, strides=2, padding=1))
net.add(nn.Flatten())
net.add(nn.Dense(4096, activation='relu'))
net.add(nn.Dropout(0.5))
net.add(nn.Dense(4096, activation='relu'))
net.add(nn.Dropout(0.5))
net.add(nn.Dense(1000))

In [417]:
#net = gluon.model_zoo.vision.alexnet(pretrained=True)
net.initialize(mx.init.Uniform(scale=1))
optim = optimizer.Adam(learning_rate=config['learning_rate'])
trainer = gluon.Trainer(net.collect_params(), optim)

In [432]:
x = net(mx.nd.zeros((1, 32, 32, 3)))
try:
    x = x.asnumpy()
except:
    x = x.asnumpy()

In [433]:
x

array([[ 0.00000000e+00,  0.00000000e+00,  2.98476573e-43,
         0.00000000e+00, -1.02578287e+34,  4.58210585e-41,
         1.34524653e-43,  0.00000000e+00,  5.88545355e-44,
         0.00000000e+00,  2.67226686e-35,  3.01808660e-29,
         1.16621059e-34,  6.91455370e-18, -2.12702664e+38,
         3.02888693e-20,  4.50236107e-31,  1.32706333e-16,
        -2.00708844e+38,  3.68880274e-14, -1.94062501e+38,
         7.71586367e-15, -2.98139341e+09,  2.36474542e+29,
         1.00349891e-04,  4.58252624e-41,  1.00349891e-04,
         4.58252624e-41, -2.22152009e-14,  9.40667067e-31,
        -2.88336968e+29,  4.58238611e-41, -2.88336968e+29,
         4.58238611e-41,  1.40129846e-42,  0.00000000e+00,
        -5.08326106e+29,  4.58238611e-41, -5.08326106e+29,
         4.58238611e-41,  0.00000000e+00,  0.00000000e+00,
         7.74249109e-10,  4.58252624e-41,  7.74249109e-10,
         4.58252624e-41, -2.43166517e-19,  5.63215725e-02,
        -5.87240914e+16,  4.58238611e-41, -5.87240914e+1

In [425]:
### AlexNet and CIFAR100 
# Net

class AlexNet(HybridBlock):
    r"""AlexNet model from the `"One weird trick..." `_ paper.

    Parameters
    ----------
    classes : int, default 1000
        Number of classes for the output layer.
    """
    def __init__(self, classes=1000, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        with self.name_scope():
            self.features = nn.HybridSequential(prefix='')
            with self.features.name_scope():
                self.features.add(nn.Conv2D(64, kernel_size=11, strides=4,
                                            padding=5, activation='relu', in_channels=3,layout="NHWC"))
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2, padding=1))
                self.features.add(nn.Conv2D(256, kernel_size=5, padding=2,
                                            activation='relu',layout="NHWC"))
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2, padding=1))
                self.features.add(nn.Conv2D(384, kernel_size=3, padding=1,
                                            activation='relu', layout="NHWC"))
                self.features.add(nn.Conv2D(256, kernel_size=3, padding=1,
                                            activation='relu', layout="NHWC"))
                self.features.add(nn.Conv2D(256, kernel_size=3, padding=1,
                                            activation='relu', layout="NHWC"))
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2, padding=1))
                self.features.add(nn.Flatten())
                self.features.add(nn.Dense(4096, activation='relu'))
                self.features.add(nn.Dropout(0.5))
                self.features.add(nn.Dense(4096, activation='relu'))
                self.features.add(nn.Dropout(0.5))

            self.output = nn.Dense(classes)

    def hybrid_forward(self, F, x):
        x = self.features(x)
        x = self.output(x)
        return x

In [318]:
def transform(data, label):
    data = data.astype('float32')/255
    return data, label

transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0, 1)])

train_dataset = vision.datasets.CIFAR100(train=True, transform=vision.transforms.Compose())
valid_dataset = vision.datasets.CIFAR100(train=False, transform=transform)

TypeError: __init__() missing 1 required positional argument: 'transforms'

In [457]:
def test(ctx, val_data, net):
    metric = mx.metric.Accuracy()
    for data, label in val_data:
        data = data.as_in_context(ctx[0])
        label = label.as_in_context(ctx[0])
        output = net(data)
        try:
            metric.update(label, output)
        except:
            metric.update(label, output)
    return metric.get()

In [458]:
def mxnet_objective(config):
    net = AlexNet()
    #net = gluoncv.model_zoo.get_model('alexnet', classes=1000, pretrained=False)
    gpus = mx.test_utils.list_gpus()
    ctx = [mx.gpu()] if gpus else [mx.cpu()]
    net.initialize(mx.init.Uniform(scale=1), ctx=ctx)
    optim = optimizer.Adam(learning_rate=config['learning_rate'])
    trainer = gluon.Trainer(net.collect_params(), optim)
    
    train_data = gluon.data.DataLoader(vision.datasets.CIFAR100(train=True, transform=transform), batch_size=config['batch_size'], shuffle=False)
    val_data = gluon.data.DataLoader(vision.datasets.CIFAR100(train=False, transform=transform), batch_size=config['batch_size'], shuffle=False)

    train_metric = mx.metric.Accuracy()
    train_history = TrainingHistory(['training-error', 'validation-error'])
    
#     # Use Accuracy as the evaluation metric.
#     metric = mx.metric.Accuracy()
    criterion = gluon.loss.SoftmaxCrossEntropyLoss()
    
        
    for epoch in range(config['epochs']):
        train_metric.reset()
        train_loss, train_acc, valid_acc = 0., 0., 0.
        for data, label in train_data:
            # forward + backward
            with ag.record():
                output = net(data)
                loss = criterion(output, label)
            loss.backward()
            # update parameters
            trainer.step(config['batch_size'])

    name, acc = train_metric.get()
    # Evaluate on Validation data
    name, val_acc = test(ctx, val_data, net)
    print(val_acc)

#     # Update history and print metrics
#     #train_history.update([1-acc, 1-val_acc])
    
# #     return (val_acc, net)
#     for epoch in range(config['epochs']):
#     # training loop (with autograd and trainer steps, etc.)
#         cumulative_train_loss = mx.nd.zeros(1)
#         training_samples = 0
#         for batch_idx, (data, label) in enumerate(train_data):
#             with ag.record():
#                 output = net(data)
#                 loss = criterion(output, label)
#             loss.backward()
#             trainer.step(data.shape[0])
#             cumulative_train_loss += loss.sum()
#             training_samples += data.shape[0]
#         try:
#             train_loss = cumulative_train_loss.asscalar()/training_samples
#         except:
#             train_loss = cumulative_train_loss.asscalar()/training_samples
#         print(cumulative_train_loss)

#     # validation loop
#     cumulative_valid_loss = mx.nd.zeros(1, ctx)
#     valid_samples = 0
#     for batch_idx, (data, label) in enumerate(val_data):
#         data = data.as_in_context(ctx).reshape((-1, 1024)) # 28*28=784
#         label = label.as_in_context(ctx)
#         output = net(data)
#         loss = criterion(output, label)
#         cumulative_valid_loss += loss.sum()
#         valid_samples += data.shape[0]
#     valid_loss = cumulative_valid_loss.asscalar()/valid_samples

#     print("Epoch {}, training loss: {:.2f}, validation loss: {:.2f}".format(epoch, train_loss, valid_loss))

In [459]:
config = {'learning_rate':.001, 'dropout':0.5, 'batch_size':100, 'epochs':1}

In [460]:
mxnet_objective(config)

0.05


In [283]:
def acc(output, label):
    # output: (batch, num_output) float32 ndarray
    # label: (batch, ) int32 ndarray
    return (output.argmax(axis=1) ==
            label.astype('float32')).mean().asscalar()

In [332]:
def mxnet_objective(config):
    net = AlexNet()
    #net = gluoncv.model_zoo.get_model('alexnet', classes=1000, pretrained=False)
#     gpus = mx.test_utils.list_gpus()
#     ctx = [mx.gpu()] if gpus else [mx.cpu()]
#     net.initialize(mx.init.Uniform(scale=1), ctx=ctx)
    net.initialize(mx.init.Uniform(scale=1))
    optim = optimizer.Adam(learning_rate=config['learning_rate'])
    trainer = gluon.Trainer(net.collect_params(), optim)
    
    train_data = gluon.data.DataLoader(vision.datasets.CIFAR100(train=True, transform=transform), batch_size=config['batch_size'], shuffle=False)
    val_data = gluon.data.DataLoader(vision.datasets.CIFAR100(train=False, transform=transform), batch_size=config['batch_size'], shuffle=False)

#     train_metric = mx.metric.Accuracy()
#     train_history = TrainingHistory(['training-error', 'validation-error'])
    
#     # Use Accuracy as the evaluation metric.
#     metric = mx.metric.Accuracy()
    softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
    
        
    for epoch in range(config['epochs']):
        train_loss, train_acc, valid_acc = 0., 0., 0.
        tic = time.time()
        for data, label in train_data:
            # forward + backward
            with ag.record():
                output = net(data)
                loss = softmax_cross_entropy(output, label)
            #print(loss)
            loss.backward()
            # update parameters
            trainer.step(config['batch_size'])
            # calculate training metrics
            train_loss += loss.mean().asscalar()
            train_acc += acc(output, label)
    # calculate validation accuracy
    for data, label in valid_data:
        valid_acc += acc(net(data), label)
    print("Epoch %d: loss %.3f, train acc %.3f, test acc %.3f, in %.1f sec" % (
            epoch, train_loss/len(train_data), train_acc/len(train_data),
            valid_acc/len(valid_data), time.time()-tic))

In [333]:
mxnet_objective(config)

MXNetError: MXNetError: could not create a descriptor for a dilated convolution forward propagation primitive

In [312]:
net = AlexNet()
#net = gluoncv.model_zoo.get_model('alexnet', classes=1000, pretrained=False)
#     gpus = mx.test_utils.list_gpus()
#     ctx = [mx.gpu()] if gpus else [mx.cpu()]
#     net.initialize(mx.init.Uniform(scale=1), ctx=ctx)
net.initialize(mx.init.Uniform(scale=1))
optim = optimizer.Adam(learning_rate=config['learning_rate'])
trainer = gluon.Trainer(net.collect_params(), optim)

train_data = gluon.data.DataLoader(vision.datasets.CIFAR100(train=True, transform=transform), batch_size=config['batch_size'], shuffle=False)
val_data = gluon.data.DataLoader(vision.datasets.CIFAR100(train=False, transform=transform), batch_size=config['batch_size'], shuffle=False)

#     train_metric = mx.metric.Accuracy()
#     train_history = TrainingHistory(['training-error', 'validation-error'])

#     # Use Accuracy as the evaluation metric.
#     metric = mx.metric.Accuracy()
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
ctx = mx.cpu(0)

In [316]:
for epoch in range(10):
    cum_loss = 0.0
    for inputs, labels in train_data:
        inputs = inputs.as_in_context(ctx)
        # ctx can be mx.gpu() if gpu available otherwise mx.cpu()
        labels = labels.as_in_context(ctx)
        # ctx can be mx.gpu() if gpu available otherwise mx.cpu()
        with ag.record():
            outputs = net(inputs)
            loss = softmax_cross_entropy(outputs, labels)
            # here you are computing the loss
        loss.backward()
        # here you are differentiating your loss with respect to all your model parameters.
        trainer.step(batch_size=inputs.shape[0])
        # running trainer
        cum_loss += loss.mean()
        # accumulating loss of current batch
    cum_loss = cum_loss.as_in_context(ctx)
    epoch_loss = cum_loss.asscalar()/len(train_loader) 
    # len(train_loader) gives number of batches, so you are dividing all accumulated loss by num of batches
    print('After epoch {}: Loss: {}'.format(epoch + 1, epoch_loss))

MXNetError: MXNetError: could not create a descriptor for a dilated convolution forward propagation primitive